In [1]:
import os
import pandas as pd
import glob
from functools import reduce

In [2]:
os.chdir("/fs/ess/PAS0439/MING/virome_ecology/results/R_project/")

In [3]:
metadata = pd.read_csv("metadata_ecology.csv")

In [4]:
samples = list(metadata.query('project == "PRJEB31266"').id)

In [ ]:
virome = pd.read_csv("/fs/ess/PAS0439/MING/virome/checkv_trimmed_for_dowmstream/ecology/abundance_table_raw.tsv", sep = "\t")

In [ ]:
virome = virome[["Contig"] + samples]

In [ ]:
# filter coverm results < 1 
def func1(x):
    if x > 1:
        return  x
    else:
        return  0
    
virome_tmp1 = virome[samples].applymap(func1)

# change to presence absence matrix
def func2(x):
    if x > 1:
        return  1
    else:
        return  0

virome_tmp2 = virome_tmp1[samples].applymap(func2)

# prevalence filtering (70% threshold)
filtered_votus_70 = list(pd.DataFrame(virome_tmp2.sum(axis='columns')).rename(columns = {0:'total_prevalence'}).query('total_prevalence >= 168').index)
filtered_votus_50 = list(pd.DataFrame(virome_tmp2.sum(axis='columns')).rename(columns = {0:'total_prevalence'}).query('total_prevalence >= 120').index)


In [ ]:
virome_filtered_70 = virome.loc[filtered_votus_70]
virome_filtered_50 = virome.loc[filtered_votus_50]

In [ ]:
os.chdir("/fs/scratch/PAS0439/Ming/virome_ecology_core_prkaryotes/results/01_kraken2_mapping/gtdb207_rumenmagsfull/bracken")

In [ ]:
microbe_bracken = [f for f in glob.glob("*_s.bracken") if f.split("_s.bracken")[0] in samples]

In [ ]:
df_list = []
for f in microbe_bracken:
    sample = f.split("_s.bracken")[0]
    _ = pd.read_csv(f, sep = "\t")[["name", "new_est_reads", "fraction_total_reads"]].query('fraction_total_reads > 0.0001')
    _.rename(columns={"new_est_reads":sample}, inplace = True)
    df_list.append(_[["name", sample]])
    
microbe_profile_pre = reduce(lambda df1,df2: pd.merge(df1,df2,on='name', how = "outer"), df_list)

In [ ]:
microbe_profile_pre.fillna(0, inplace = True)

In [ ]:
# prevalence filtering (70/50 % threshold)
species_filtered_70 = []
species_filtered_50 = []
for index, row in microbe_profile_pre.iterrows():
    tmp = row
    tmp = pd.DataFrame(tmp)
    tmp.columns = tmp.iloc[0]
    species = tmp.columns[0]
    tmp = tmp.iloc[1:,:]
    tmp.rename(columns = {species:"species"}, inplace = True)
    tmp = tmp.query('species > 0')
    prevalence = len(tmp)
    
    if prevalence >= 0.7*len(samples):
        species_filtered_70.append(species)
        species_filtered_50.append(species)
    elif prevalence >= 0.5*len(samples):
        species_filtered_50.append(species)

In [ ]:
microbe_profile_filtered_70 = microbe_profile_pre.query('name in @species_filtered_70')
microbe_profile_filtered_70 = microbe_profile_filtered_70.set_index("name")

microbe_profile_filtered_50 = microbe_profile_pre.query('name in @species_filtered_50')
microbe_profile_filtered_50 = microbe_profile_filtered_50.set_index("name")

In [ ]:
virome_filtered_70.set_index("Contig")[samples].transpose().to_csv("/fs/ess/PAS0439/MING/virome_ecology/results/R_project/virome_microbes_network_RUG4941/virome_prevalence_filtered_70.csv")
microbe_profile_filtered_70[samples].transpose().to_csv("/fs/ess/PAS0439/MING/virome_ecology/results/R_project/virome_microbes_network_RUG4941/microbes_prevalence_filtered_70.csv")

virome_filtered_50.set_index("Contig")[samples].transpose().to_csv("/fs/ess/PAS0439/MING/virome_ecology/results/R_project/virome_microbes_network_RUG4941/virome_prevalence_filtered_50.csv")
microbe_profile_filtered_50[samples].transpose().to_csv("/fs/ess/PAS0439/MING/virome_ecology/results/R_project/virome_microbes_network_RUG4941/microbes_prevalence_filtered_50.csv")